<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 6. Redes Neuronales Recurrentes</font></h1>

<h1><font color="#113D68" size=4>6. Práctica: Procesamiento del Lenguaje Natural</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Descripción del problema: generación de texto](#section1)
* [2. LSTM de linea base](#section2)
    * [2.1. Cargar el dataset](#section2.1)
    * [2.2. Conversión a numérico](#section2.2)
    * [2.3. Dimensiones del dataset](#section2.3)
    * [2.4. Procesamiento de datos](#section2.4)
    * [2.5. Diseño de la LSTM](#section2.5)
    * [2.6. Crear puntos de control](#section2.6)
    * [2.7. Resultados](#section2.7)
* [3. Generación de texto con LSTM](#section3)
    * [3.1. Cargar los pesos de LSTM](#section3.1)
    * [3.2. Convertir de entero a carácter](#section3.2)
    * [3.3. Resultados y evaluación](#section3.3)
* [4. LSTM más profunda](#section4)
* [5. Mejorar nuestro modelo](#section5)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

Las redes neuronales recurrentes también se pueden utilizar como modelos generativos. Esto significa que, además de usarse para modelos predictivos (hacer predicciones), pueden aprender las secuencias de un problema y luego generar secuencias plausibles completamente nuevas para el dominio del problema. Los modelos generativos como este son útiles no solo para estudiar qué tan bien un modelo ha aprendido un problema, sino para aprender más sobre el dominio del problema en sí. 

En este proyecto, descubrirás cómo crear un modelo generativo de texto, carácter por carácter, utilizando redes neuronales recurrentes LSTM. Después de completar este proyecto, sabrá:
* Dónde descargar un corpus de texto gratuito que puede utilizar para entrenar modelos generativos de texto.
* Cómo enmarcar el problema de las secuencias de texto a un modelo generativo de redes neuronales recurrentes.
* Cómo desarrollar un LSTM para generar secuencias de texto plausibles para un problema dado.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Descripción del problema: generación de texto</font>

Muchos de los textos clásicos ya no están protegidos por derechos de autor. Esto significa que puede descargar todo el texto de estos libros de forma gratuita y utilizarlos en experimentos, como la creación de modelos generativos. Quizás el mejor lugar para acceder a libros gratuitos que ya no están protegidos por derechos de autor es el Proyecto Gutenberg. Para este proyecto vamos a utilizar el libro "Alicia en el país de las maravillas" de Lewis Carroll.

Vamos a aprender las dependencias entre personajes y las probabilidades condicionales de personajes en secuencias para que a su vez podamos generar secuencias de personajes totalmente nuevas y originales. Este tutorial es muy divertido y recomiendo repetir estos experimentos con otros libros del Proyecto Gutenberg. Estos experimentos no se limitan al texto, también puede experimentar con otros datos ASCII, como código fuente de computadora, documentos marcados en LaTeX, HTML o Markdown y más.

Puede descargar el texto completo en formato ASCII (Texto sin formato UTF-8) de este libro de forma gratuita y colocarlo en su directorio de trabajo con el nombre `wonderland.txt`. 

El Proyecto Gutenberg agrega un encabezado y un pie de página estándar a cada libro y esto no es parte del texto original. Abra el archivo en un editor de texto y elimine el encabezado y el pie de página. El encabezado es obvio y termina con el texto:
```
        *** START OF THIS PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***
```

El pie de página es todo el texto después de la línea de texto que dice:
```
        THE END
```


Debería quedarse con un archivo de texto que tiene aproximadamente 3330 líneas de texto.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el [Proyecto Gutenberg](https://www.gutenberg.org/)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Descargar el libro [Alice's Adventures in Wonderland](https://www.gutenberg.org/ebooks/11)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. LSTM de linea base</font>

En esta sección desarrollaremos una red LSTM simple para aprender secuencias de caracteres de Alicia en el país de las maravillas. En la siguiente sección usaremos este modelo para generar nuevas secuencias de caracteres. 

<a id="section2.1"></a>
# <font color="#004D7F" size=5>2.1. Cargar el dataset</font>

Comencemos importando las clases y funciones que pretendemos usar para entrenar nuestro modelo.

A continuación, debemos cargar el texto ASCII del libro en la memoria y convertir todos los caracteres a minúsculas para reducir el vocabulario que la red debe aprender.

In [17]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "data/wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

<a id="section2.2"></a>
# <font color="#004D7F" size=5>2.2. Conversión a numérico</font>

Ahora que el libro está cargado, debemos preparar los datos para modelarlos mediante la red neuronal. No podemos modelar los caracteres directamente, sino que debemos convertir los caracteres a números enteros. Podemos hacer esto fácilmente creando primero un conjunto de todos los caracteres distintos en el libro, luego creando un mapa de cada personaje a un número entero único.

Por ejemplo, la lista de caracteres en minúscula ordenados únicos en el libro es la siguiente:

In [18]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 'ù': 58,
 '—': 59,
 '‘': 60,
 '’': 61,
 '“': 62,
 '”': 63,
 '\ufeff': 64}

Puede ver que puede haber algunos caracteres que podríamos eliminar para limpiar aún más el conjunto de datos, lo que reducirá el vocabulario y puede mejorar el proceso de modelado. 

<a id="section2.3"></a>
# <font color="#004D7F" size=5>2.3. Dimensiones del dataset</font>

Ahora que se cargó el libro y se preparó el mapeo, podemos resumir el conjunto de datos.

In [19]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  164202
Total Vocab:  65


Podemos ver que el libro tiene poco menos de 150.000 caracteres y que cuando se convierte a minúsculas sólo hay 47 caracteres distintos en el vocabulario para que la red los aprenda. Mucho más que los 26 del alfabeto. 

Ahora necesitamos definir los datos de entrenamiento para la red. Hay mucha flexibilidad en la forma en que elige dividir el texto y exponerlo a la red durante el entrenamiento. En este tutorial, dividiremos el texto del libro en subsecuencias con una longitud fija de 100 caracteres, una longitud arbitraria. Podríamos dividir fácilmente los datos en oraciones y rellenar las secuencias más cortas y truncar las más largas.

Cada patrón de entrenamiento de la red se compone de 100 pasos de tiempo de un carácter (X) seguidos de una salida de carácter (y). Al crear estas secuencias, deslizamos esta ventana a lo largo de todo el libro, un carácter a la vez, permitiendo que cada carácter tenga la oportunidad de aprender de los 100 caracteres que lo precedieron (excepto los primeros 100 caracteres, por supuesto). Por ejemplo, si la longitud de la secuencia es 5 (para simplificar), los dos primeros patrones de entrenamiento serían los siguientes:
```
            CHAPT -> E
            HAPTE -> R
```



A medida que dividimos el libro en estas secuencias, convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente.

In [20]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  164102


Ejecutar el código hasta este punto nos muestra que cuando dividimos el conjunto de datos en datos de entrenamiento para que la red sepa que tenemos poco más de 150.000 patrones de entrenamiento. Esto tiene sentido ya que excluyendo los primeros 100 caracteres, tenemos un patrón de entrenamiento para predecir cada uno de los caracteres restantes.

<a id="section2.4"></a>
# <font color="#004D7F" size=5>2.4. Procesamiento de datos</font>

Ahora que hemos preparado nuestros datos de entrenamiento, necesitamos transformarlos. 
1. Primero debemos transformar la lista de secuencias de entrada en la forma [muestras, pasos de tiempo, características] esperada por una red LSTM. 
2. A continuación, debemos cambiar la escala de los números enteros al rango de 0 a 1 para que los patrones sean más fáciles de aprender mediante la red LSTM que usa la función de activación sigmoidea de forma predeterminada.
3. Finalmente, necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) con One-Hot Encoding. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 47 caracteres diferentes en el vocabulario (una representación más fácil) en lugar de intentar forzarlo a predecir con precisión el siguiente carácter. Cada valor de _y_ se convierte en un vector disperso con una longitud de 47, lleno de ceros, excepto con un 1 en la columna de la letra (número entero) que representa el patrón. 

In [21]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

<a id="section2.5"></a>
# <font color="#004D7F" size=5>2.5. Diseño de la LSTM</font>

Ahora podemos definir nuestro modelo LSTM. 
1. Definimos una única capa LSTM oculta con 256 unidades de memoria. 
2. La red utiliza un dropout con una probabilidad del 20%. 
3. La capa de salida es una capa densa que utiliza la función de activación Softmax para generar una predicción de probabilidad para cada uno de los 47 caracteres entre 0 y 1. 
4. El problema es realmente un problema de clasificación de un solo carácter con 47 clases y, como tal, se define como la optimización de la pérdida logarítmica (`categorical_crossentropy`)
5. Usaremos el algoritmo de optimización de Adam.

In [22]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

No hay un conjunto de datos de prueba. Estamos modelando todo el conjunto de datos de entrenamiento para conocer la probabilidad de cada personaje en una secuencia. 

No estamos interesados en el modelo más preciso (Accuracy de clasificación) del conjunto de datos de entrenamiento. Este sería un modelo que predice perfectamente cada carácter en el conjunto de datos de entrenamiento. 

En cambio, estamos interesados en una generalización del conjunto de datos que minimice la función de pérdida elegida. Buscamos un equilibrio entre la generalización y el sobreajuste, pero sin memorizar.

<a id="section2.6"></a>
# <font color="#004D7F" size=5>2.6. Crear puntos de control</font>

La red es lenta de entrenar. Debido a la lentitud y debido a nuestros requisitos de optimización, utilizaremos puntos de control de modelo para registrar todos los pesos de la red para archivar cada vez que se observe una mejora en la pérdida al final de la época. Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección.

In [23]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

<a id="section2.7"></a>
# <font color="#004D7F" size=5>2.7. Resultados</font>

Ahora podemos ajustar nuestro modelo a los datos. Aquí utilizamos un número modesto de 20 épocas y un gran tamaño de batch de 128 patrones.

In [ ]:
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1283/1283 [==============================] - ETA: 0s - loss: 3.0190
Epoch 00001: loss improved from inf to 3.01901, saving model to weights-improvement-01-3.0190.hdf5
1283/1283 [==============================] - 301s 235ms/step - loss: 3.0190
Epoch 2/20
1283/1283 [==============================] - ETA: 0s - loss: 2.8445
Epoch 00002: loss improved from 3.01901 to 2.84446, saving model to weights-improvement-02-2.8445.hdf5
1283/1283 [==============================] - 297s 231ms/step - loss: 2.8445
Epoch 3/20
1283/1283 [==============================] - ETA: 0s - loss: 2.7635
Epoch 00003: loss improved from 2.84446 to 2.76349, saving model to weights-improvement-03-2.7635.hdf5
1283/1283 [==============================] - 299s 233ms/step - loss: 2.7635
Epoch 4/20
1033/1283 [=======================>......] - ETA: 56s - loss: 2.7060

Después de ejecutar el ejemplo, debería tener varios archivos de puntos de control de peso en el directorio local. Puede eliminarlos todos excepto el que tenga el valor de pérdida más pequeño. 

La pérdida de red disminuyó casi en todas las épocas y espero que la red pueda beneficiarse del entrenamiento durante muchas más épocas. En la siguiente sección veremos el uso de este modelo para generar nuevas secuencias de texto.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Generación de texto con LSTM</font>

La generación de texto utilizando la red LSTM entrenada es relativamente sencilla. 

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Cargar los pesos de LSTM</font>

En primer lugar, cargamos los datos y definimos la red exactamente de la misma manera, excepto que los pesos de la red se cargan desde un archivo de punto de control y no es necesario entrenar la red. 

In [ ]:
# load the network weights
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Convertir de entero a carácter</font>

Además, al preparar el mapeo de caracteres únicos a números enteros, también debemos crear un mapeo inverso que podamos usar para convertir los números enteros de nuevo en caracteres para que podamos entender las predicciones.

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data

<a id="section3.3"></a>
# <font color="#004D7F" size=5>3.3. Resultados y evaluación</font>

Finalmente, necesitamos realmente hacer predicciones. 

La forma más sencilla de utilizar el modelo Keras LSTM para hacer predicciones es comenzar primero con una semilla como entrada, generar el siguiente carácter y luego actualizar la semilla para agregar el carácter generado al final y recortar el primer carácter. Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1000 caracteres de longitud). 

Podemos elegir un patrón de entrada aleatorio como nuestra secuencia semilla, luego imprimir los caracteres generados a medida que los generamos.

In [ ]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Al ejecutar este ejemplo, primero se genera la semilla aleatoria seleccionada, luego cada carácter a medida que se genera. 

Podemos notar algunas observaciones sobre el texto generado.
* Generalmente se ajusta al formato de línea observado en el texto original de menos de 80 caracteres antes de una nueva línea.
* Los caracteres están separados en grupos parecidos a palabras y la mayoría de los grupos son palabras reales en inglés (por ejemplo, _the, little_ y _was),_ pero muchos no (por ejemplo, _lott, tiie_ y _taede)._
* Algunas de las palabras en secuencia tienen sentido (por ejemplo, y el _white rabbit),_ pero muchas no (por ejemplo, _wese tilel)._

El hecho de que este modelo del libro basado en caracteres produzca resultados como este es muy impresionante. Le da una idea de las capacidades de aprendizaje de las redes LSTM. Los resultados no son perfectos. 

En la siguiente sección, veremos cómo mejorar la calidad de los resultados mediante el desarrollo de una red LSTM mucho más grande.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. LSTM más profunda</font>

Ahora, podemos intentar mejorar la calidad del texto generado creando una red mucho más grande. Mantendremos el mismo número de unidades de memoria en 256, pero agregaremos una segunda capa.

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

También cambiaremos el nombre de archivo de los pesos con puntos de control para que podamos distinguir entre los pesos de esta red y la anterior (agregando la palabra más grande en el nombre del archivo).

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Finalmente, aumentaremos el número de épocas de entrenamiento de 20 a 50 y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender. 

In [ ]:
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Después de ejecutar este ejemplo, puede lograr una pérdida de aproximadamente 1,2. 

Logrando una pérdida de 1.2219 en la época 47. Como en la sección anterior, podemos usar este mejor modelo de la ejecución para generar texto. El único cambio que debemos realizar en el script de generación de texto de la sección anterior es la especificación de la topología de la red y desde qué archivo se van a generar los pesos de la red. 

In [ ]:
# load the network weights
filename = "weights-improvement-47-1.2219-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Podemos ver que en general hay menos errores de ortografía y el texto parece más realista, pero sigue siendo bastante absurdo. Por ejemplo las mismas frases se repiten una y otra vez. 

Estos son mejores resultados, pero todavía hay mucho margen de mejora.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section5"></a>
# <font color="#004D7F" size=6>5. Mejorar nuestro modelo</font>

A continuación, se muestra una muestra de ideas que tal vez desee investigar para mejorar aún más el modelo:
* Predecir menos de 1000 caracteres como salida para una semilla determinada.
* Eliminar toda la puntuación del texto fuente y, por tanto, del vocabulario de los modelos.
* Pruebe un One-Hot Encoding para las secuencias de entrada.
* Entrene al modelo en oraciones rellenas en lugar de secuencias aleatorias de caracteres.
* Aumentar el número de épocas de entrenamiento a 100 o más.
* Agregue Dropout a la capa de entrada visible y considere ajustar el porcentaje de Dropout.
* Ajuste el tamaño de batch, pruebe con un tamaño de batch de 1 como línea de base (muy lenta) y tamaños más grandes a partir de ahí.
* Agregue más unidades de memoria a las capas y / o más capas.
* Cambie las capas de LSTM para que tengan estado para mantener el estado en todos los batch.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>